### BioPython: GenBank Data Extraction & Disease Variant Identification 

Hereditary hemochromatosis is an autosomal recessive disease in which the body’s iron stores are increased, with serious negative effects on the function of several organs (liver cirrhosis, diabetes mellitus, heart failure among others). This disease has an unusually high incidence rate among the settled populations of north western europe, most especially in Ireland, where the incidence rate is estimated to be almost 1 in 100. Although a complex disorder, the the HFE gene is known to play a dominant role in hereditary hemochromatosis. Most patients with manifest hereditary hemochromatosis are homozygous for the C282Y (Cys282Tyr) mutation (https://www.snpedia.com/index.php/Rs1800562), a transition point mutation from guanine to adenine (G -> A) at nucleotide 845 in the gene's coding sequence - this results in a missense mutation, replacing the cysteine residue at position 282 with a tyrosine amino acid. This change disrupts correct formation of the HFE protein, interfering with its ability to regulate iron intake, in effect resulting in an increase in iron, with associated deleterious effects.

The reference gene sequence for the human variant of HFE in the GenBank archive has the accession code NG_008720 - this includes all coding and non-coding sequences annotated at this genomic locus in chromosome 6. 

*Use BioPython utilities to:*

#### 1. *Download the SeqRecord for this accession.*

The sequence record is got through a data parser. The GenBank information for HFE is obtained by using Entrez.

Entrez is a search and retrival system used by NCBI to search major databases such as PubMed, nucleotide and protein sequences, taxonomy, etc. 
        
- "efetch" retrieves records in the requested format from a list of one or more primary IDs or from the user's environment
- "db" specifies the database to search in NCBI.
- "rettype" specifies the retrival type and can be either uilist or count.
- "retmode" determines the format of the returned output.
- "id" is used to find the individual gene.

In [9]:
from Bio import Entrez
from Bio import SeqIO

try:

    Entrez.email = "A.N.Other@example.com"   #dummy email generated to avoid error
    
    #
    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NG_008720") # handle is our link to NCBI

    HFE = SeqIO.read(handle, "gb") #variable coordinating with HFE and obtained through NCBI database search in GenBank

    handle.close()

    print(type(HFE))  # connection closed. now confirm it is a sequence record.

except:

    print('Failed to fetch record.') 

<class 'Bio.SeqRecord.SeqRecord'>


You can now access the ID and name by using the HDE

In [3]:
print(HFE.id) # pulls the ID from the NCBI website.
print(HFE.name) # pulls the name from the NCBI website.

NG_008720.2
NG_008720


#### 2. *Extract the coding sequence for the HFE gene from the GenBank record (that is the concantenated exon sequences).*

By calling the features, we can obtain information about exons, mRNA and the coding sequence (CDS).  

In [4]:
for i in HFE.features: 
    print(i)

type: source
location: [0:18063](+)
qualifiers:
    Key: chromosome, Value: ['6']
    Key: db_xref, Value: ['taxon:9606']
    Key: map, Value: ['6p22.2']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Homo sapiens']

type: gene
location: [4009:8754](-)
qualifiers:
    Key: db_xref, Value: ['GeneID:108783645']
    Key: gene, Value: ['LOC108783645']
    Key: note, Value: ['HFE antisense RNA']

type: ncRNA
location: join{[8442:8754](-), [4009:5882](-)}
qualifiers:
    Key: db_xref, Value: ['GeneID:108783645']
    Key: gene, Value: ['LOC108783645']
    Key: ncRNA_class, Value: ['antisense_RNA']
    Key: product, Value: ['HFE antisense RNA']
    Key: transcript_id, Value: ['NR_144383.1']

type: gene
location: [5000:12961](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:3077', 'HGNC:HGNC:4886', 'MIM:613609']
    Key: gene, Value: ['HFE']
    Key: gene_synonym, Value: ['HFE1; HH; HLA-H; MVCD7; TFQTL2']
    Key: note, Value: ['homeostatic iron regulator']

type: mRNA
lo

In [200]:
print(type(HFE.features))

<class 'list'>


From the features information that is obtained from GenBank, I can use the exon locations and concatenate them together. The coding region locations are found from the "CDS" section. They are concatenated together using a for loop. 

In [5]:
coding = ([5160,5236], [8560,8824], [9033,9309], [10404,10680], [10838,10952], [11905,11946])

conc_exons = ""
for i in coding:
    exon = HFE.seq[i[0]:i[1]]
    conc_exons = conc_exons + exon
print(conc_exons)

ATGGGCCCGCGAGCCAGGCCGGCGCTTCTCCTCCTGATGCTTTTGCAGACCGCGGTCCTGCAGGGGCGCTTGCTGCGTTCACACTCTCTGCACTACCTCTTCATGGGTGCCTCAGAGCAGGACCTTGGTCTTTCCTTGTTTGAAGCTTTGGGCTACGTGGATGACCAGCTGTTCGTGTTCTATGATCATGAGAGTCGCCGTGTGGAGCCCCGAACTCCATGGGTTTCCAGTAGAATTTCAAGCCAGATGTGGCTGCAGCTGAGTCAGAGTCTGAAAGGGTGGGATCACATGTTCACTGTTGACTTCTGGACTATTATGGAAAATCACAACCACAGCAAGGAGTCCCACACCCTGCAGGTCATCCTGGGCTGTGAAATGCAAGAAGACAACAGTACCGAGGGCTACTGGAAGTACGGGTATGATGGGCAGGACCACCTTGAATTCTGCCCTGACACACTGGATTGGAGAGCAGCAGAACCCAGGGCCTGGCCCACCAAGCTGGAGTGGGAAAGGCACAAGATTCGGGCCAGGCAGAACAGGGCCTACCTGGAGAGGGACTGCCCTGCACAGCTGCAGCAGTTGCTGGAGCTGGGGAGAGGTGTTTTGGACCAACAAGTGCCTCCTTTGGTGAAGGTGACACATCATGTGACCTCTTCAGTGACCACTCTACGGTGTCGGGCCTTGAACTACTACCCCCAGAACATCACCATGAAGTGGCTGAAGGATAAGCAGCCAATGGATGCCAAGGAGTTCGAACCTAAAGACGTATTGCCCAATGGGGATGGGACCTACCAGGGCTGGATAACCTTGGCTGTACCCCCTGGGGAAGAGCAGAGATATACGTGCCAGGTGGAGCACCCAGGCCTGGATCAGCCCCTCATTGTGATCTGGGAGCCCTCACCGTCTGGCACCCTAGTCATTGGAGTCATCAGTGGAATTGCTGTTTTTGTCGTCATCTTGTTCATTGGAATTTTGTTCATAATATTAAGGAAGAGGCAGG

#### *3. Use the .translate() method to create the amino acid sequence from your determined full sequence and compare your estimate with the amino acid sequence available in the GenBank record. Demonstrate they agree!*

The final nucleotide is removed from the sequence of concatenated amino acids for comparison. 

In [36]:
HFE_protein = conc_exons.translate()
HFE_protein = HFE_protein[:-1] # -1 to remove the ambiguous nucletide
print(HFE_protein) 

MGPRARPALLLLMLLQTAVLQGRLLRSHSLHYLFMGASEQDLGLSLFEALGYVDDQLFVFYDHESRRVEPRTPWVSSRISSQMWLQLSQSLKGWDHMFTVDFWTIMENHNHSKESHTLQVILGCEMQEDNSTEGYWKYGYDGQDHLEFCPDTLDWRAAEPRAWPTKLEWERHKIRARQNRAYLERDCPAQLQQLLELGRGVLDQQVPPLVKVTHHVTSSVTTLRCRALNYYPQNITMKWLKDKQPMDAKEFEPKDVLPNGDGTYQGWITLAVPPGEEQRYTCQVEHPGLDQPLIVIWEPSPSGTLVIGVISGIAVFVVILFIGILFIILRKRQGSRGAMGHYVLAERE


From the SeqRecord sequence, the translation of the coding sequence is obtained. A conditional "if" is created to recognise whether the generated translated sequence and the database obtained translated sequence are the same. 

In [37]:
for i in HFE.features[6].qualifiers['translation']: # 6 contains the CDS 
    db_trans = i # assign the database obtained translation to i 
    if db_trans == HFE_protein: 
        print('The translated sequence for HFE matches the GenBank sequence!')
    else: 
        print('The translated sequence for HFE does not match the GenBank sequence!')
        

The translated sequence for HFE matches the GenBank sequence!


#### *4. Use this coding sequence to determine what the status is of this locus in this reference gene record for HEF (i.e. what is the nucleotide value at position 845 in HEF's coding sequence).*

Position 845 is obtained by calling 844 as R starts counting at 0. The nucleotide guanine indicates a normal nucleotide. The nucleotide adenine indicates a point mutation associated with hereditary hemochromatosis. 

In [35]:
if conc_exons[844] == "G": # check if healthy
    print("The nucleotide at this position is is guanine and is healthy.")
if conc_exons[844] == "A": # check if missense mutation
    print("The nucleotide at this position is adenine and is hereditary hemochromatosis causing.")

The nucleotide at this position is is guanine and is healthy.


#### *5. Determine which of the exons associated with the HEF gene contains the C282Y locus (i.e. is the first, second?).*

The for loop from earlier can be used again but this time include a counter. The counter will return the exon number for the region that contains missense coded protein. To thi

In [68]:
conc_exons = ''  #
count = 0
for i in coding:
    exon = HFE.seq[i[0]:i[1]]
    conc_exons = conc_exons + exon
    count = count + 1
    if len(conc_exons) in range(800,900):        
        print("The exon associated with the C282Y locus is exon number", count)

The exon associated with the C282Y locus is exon number 4


#### *6. Determine which restriction enzymes are suitable for disease variant testing.*

*One of the standard ways in which such disease variants are identified is by the use of PCR & restriction enzymes - in short a primer is prepared nearby to the proposed SNP site, and the amplified PCR products, typically 100-200 bases in length, are digested with an enzyme specific to the SNP location. The difference in fragment populations is then used to segment wild type from heterozygous/homozygous populations. Using the CDS sequence you have extracted from the GenBank record, create two amplicon sequences of length 100 bases either side of the C282Y nucleotide location, with one having the 'G' variant and the other the disease 'A' variant. For this final question you are to take both 'amplicon' sequences and to determine if any of the following restriction enzymes would be suitable for such a test (where the '/' character corresponds to where the cleaving occurs in the DNA:*
    - HinfI (G/ANTC)
    - RsaI (GT/AC)
    - EcoRI (G/AATTC) 

**For the original sequence:**

First, import the required BioPython facilites to test restriction sites for suitability. Create an amplicon 200 base pairs in length for use with the restriction model sites. Convert to a string. Determine how each restriction sites works. 

In [69]:
from Bio import Restriction
from Bio.Restriction import *
from Bio.Seq import Seq
from Bio.Alphabet.IUPAC import IUPACAmbiguousDNA
amb = IUPACAmbiguousDNA()

amplicon1 =  conc_exons[744:944] # amplicon of healthy sequence
strexons = str(amplicon1) # convert to string for testing suitability 
my_seq = Seq(strexons, amb) 

# EcoRI sites 
if len(EcoRI.search(my_seq)) > 0:
    print(len(EcoRI.search(my_seq)), "EcoRI site(s) found in the normal variant sequence at position ", EcoRI.search(my_seq))
else:
    print("No EcoRI sites found in the normal variant sequence.")

# RsaI sites
if len(RsaI.search(my_seq)) > 0:
    print(len(RsaI.search(my_seq)), "RsaI site(s) found in the normal variant sequence at position(s)", RsaI.search(my_seq))
else:
    print("No RsaI sites found in the normal variant sequence.")

if len(HinfI.search(my_seq)) > 0:
    print(len(HinfI.search(my_seq)), "HinfI site(s) found in the normal variant sequence at position(s)", HinfI.search(my_seq))
else:
    print("No HinfI site(s) found in the normal variant sequence.")


No EcoRI sites found in the normal variant sequence.
1 RsaI site(s) found in the normal variant sequence at position(s) [72]
1 HinfI site(s) found in the normal variant sequence at position(s) [180]


**For the mutated sequence:**

Create a new sequence variable with the mutatation nucleotide. Do this by converting to a list and then testing each one for restriction sites. 

In [71]:
conc_exons = list(conc_exons) # convert to a list to change the nucleotide
conc_exons[844] = "A" # convert from G to A
mutated = "".join(conc_exons)  # rejoin as a string again


amplicon2 =  mutated[744:944] # amplicon of disease sequence
print(amplicon2[100]) # confirm mutated nucleotide in correct position

strexons = str(amplicon2) # convert to string
my_seq = Seq(strexons, amb) # use function to convert to sequence and use universal sequences for ambigous DNA

if len(EcoRI.search(my_seq)) > 0:
    print(len(EcoRI.search(my_seq)), "EcoRI sites found in the mutated variant containing sequence at position ", EcoRI.search(my_seq))
else:
    print("No EcoRI sites found in the mutated variant containing sequence.")

# RsaI sites
if len(RsaI.search(my_seq)) > 0:
    print(len(RsaI.search(my_seq)), "RsaI sites found in the mutated variant containing sequence at position(s)", RsaI.search(my_seq))
else:
    print("No RsaI sites found in the mutated variant containing sequence.")

if len(HinfI.search(my_seq)) > 0:
    print(len(HinfI.search(my_seq)), "HinfI sites found in the mutated variant containing sequence at position(s)", HinfI.search(my_seq))
else:
    print("No HinfI site(s) found in the mutated variant containing sequence.")
 

A
No EcoRI sites found in the mutated variant containing sequence.
2 RsaI sites found in the mutated variant containing sequence at position(s) [72, 101]
1 HinfI sites found in the mutated variant containing sequence at position(s) [180]


The RsaI site is suitable as the restriction enzyme to use as it can cut the sequence at the region of the mutated nucleotide. The EcoRI and HinfI sites do not show any indication as to where the cut is which indicates that these would not be suitable. 